# Добро пожаловать!

Здравствуйте! Этот файл поможет вам собрать коллекцию публикаций из TikTok.

Можно собирать:
* коллекцию #Stitch-видео
* видео из конкретного аккаунта
* видео по тэгу
* видео по собранному вручную списку ссылок на TikTok-видео

Код собирает:
* Уникальные характеристики видео (csv-таблица)
* mp4-видеофайлы
* моно-wav-аудиофайлы
* релевантные тэги (csv-таблица)
* граф релевантных тэгов (gexf-файл)

Для начала запустите стартовую ячейку для импорта библиотек и инициализации функций.

Функция comments_parse_upd не работает ввиду изменения условий доступа к API со стороны TikTok.

# Стартовая ячейка

In [1]:
import os
import re
import ast
import csv
import json
import time
import requests
import winsound
import pandas as pd
import networkx as nx
from tqdm import tqdm
from numpy import random
import moviepy.editor as mp
from bs4 import BeautifulSoup
from pydub import AudioSegment
from selenium import webdriver as wb


def get_aweme_id_final (referer):
    
    ''' Функция принимает на вход ссылку на видео в тиктоке и возвращает его уникальный идентификатор (aweme_id). 
    Отделяет доменное имя и всё вплоть до слова video — и следующие после aweme_id атрибуты, например, язык.
    Подходят и ссылки для ПК, и ссылки на мобильные версии видео.'''
    
    if 'video' in referer:
        aweme_id = int (re.split('video/|\?lang|%3', referer)[1])
    
    else:
        aweme_id = int (re.split('m.tiktok.com/v/|\.html', referer)[1])
    return aweme_id


def stitch_treasury (nickname):
    
    ''' Функция принимает на вход имя пользователя, для видео которого мы хотим собрать ститчи и тэги. 
    
    Результат работы функции — словарь: {id : [подпись к ссылке/ видео, ссылка на тэг/ видео]}'''

    links_treasury = {}

    start = 0
    link_num = 0

    while True:
        print ('Стартовая позиция: ', start)
        sleeptime = random.uniform(1, 5)
        url = f'https://www.google.com/search?q=site:http://tiktok.com+intext:%23stitch+%40{nickname}_&start={start}'
        response = requests.get(url) # загружаем DOM ссылки
        soup = BeautifulSoup(response.text, 'lxml') # загружаем DOM ссылки
        resultset = soup.find_all('a') # отсортировываем только ссылки и сопровождающую их информацию
        print ('Загрузили')
        # print (url, resultset[0])
        break_count = 0
        for i in resultset:
            post_text = i.find('h3')
            link_raw = i['href']
            # print (url, post_text, link_raw)
            # print (break_count)
            if post_text is not None:
                print (post_text.text)
                pure_link = re.split('q=|&sa=', link_raw)[1] # выделяем саму ссылку из замусоренного URL
                pure_link = re.sub(r"%40", "@", pure_link)
                print (pure_link)
                link_num = link_num + 1
                links_treasury [link_num] = [post_text.text, pure_link]
                break_count = break_count + 1
        if break_count == 0:
            print ('Код завершил свою работу :)')
            return links_treasury
            break
        else:
            start = start + 10
            print ('Спит', sleeptime, 'секунд')
            time.sleep (sleeptime)
            print ('Просыпаемся! :)')
            
            
def is_it_a_video (stitch_treasury_dict_result):
    
    ''' Функция принимает на вход словарь — результат работы функции stitch_treasury — и сортирует его по типу ссылки. 
    
    Возвращает два словаря — с ссылками на видео и ссылками на тэги.
    
    "For the love, for laughter I flew up to your arms.
    Is it a video? Visions of Gideon"'''
    
    tag_num = 0
    vid_num = 0
    tag_dict = {}
    vid_dict = {}
    
    for i in stitch_treasury_dict_result.items():
        #print (i)
        if '/amp/tag/' in i[1][1]:
            # print ('tag', i[1][1])
            tag_dict [tag_num] = [i[1][0], i[1][1]]
            tag_num = tag_num + 1
        else:
            # print ('video', i[1][1])
            vid_dict [vid_num] = [i[1][0], i[1][1]]
            vid_num = vid_num + 1
    return tag_dict, vid_dict



def unique_tags_only (tags_visions_of_gideon):
    
    ''' Функция принимает на вход результат работы функции is_it_a_video, то есть словарь tags_visions_of_gideon с тэгами.
    Возвращает список уникальных очищенных тэгов-строк.'''
    
    taglist = []
    for i in tags_visions_of_gideon.items():
        # print (re.split('amp/tag/', i[1][1])[1])
        taglist.append (re.split('amp/tag/', i[1][1])[1])
    taglist = list (set (taglist))
    return taglist



def unique_list_of_links_nickname (stitch_treasury_links_nickname):
    
    ''' Функция принимает на вход результат работы функции stitch_treasury, 
    а именно словарь ссылок, и возвращает список уникальных чистых ссылок '''
    
    list_of_links = []
    for i in stitch_treasury_links_nickname.values():
        list_of_links.append (i[1])
    list_of_links = list (set (list_of_links))
    return list_of_links



def comments_parse_upd (referer):
    
    ''' Функция принимает на вход ссылку на видео в тиктоке ПК-типа (не мобильную) и собирает комментарии к этому видео. 
    Возвращает: 1 — json-результат, 2 — pandas dataframe с теми же данными.'''
    
    all_responses = dict ()

    ids = 0
    cursor = 0
    count = 50
    offset = 50
    aweme_id = get_aweme_id_final (referer) # id видео: взять из URL после video/, но до ?

    while True:
        
#         url = f"https://www.tiktok.com/api/comment/list/?aid=1988&app_name=tiktok_web&device_platform=web_pc&referer=&root_referer=&user_agent=Mozilla%2F5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&cookie_enabled=true&screen_width=1280&screen_height=1024&browser_language=ru-RU&browser_platform=Win32&browser_name=Mozilla&browser_version=5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&browser_online=true&ac=4g&timezone_name=Europe%2FMoscow&page_referer=https:%2F%2Fwww.tiktok.com%2F@russiagreecedata%3Flang%3Dru-RU&priority_region=RU&verifyFp=verify_kmxgwki8_Fxz90IPP_XZwi_4XQs_99TG_AfLnxuB6PQj0&appId=1233&region=RU&appType=m&isAndroid=false&isMobile=false&isIOS=false&OS=windows&did=6926967686699697670&tt-web-region=RU&uid=6890125463646422021&insert_ids=&aweme_id={aweme_id}&cursor={cursor}&count={count}&app_language=ru&current_region=RU&fromWeb=1&channel_id=0&_signature=_02B4Z6wo00901--3IhgAAIDBM.NfazFX5WPvtyaAAJuo33"
        
#         payload={}

#         headers = {
#             'authority': 'www.tiktok.com', # Оставить
#             'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"', # ОБНОВИТЬ
#             'accept': 'application/json, text/plain, */*', # Оставить
#             'sec-ch-ua-mobile': '?0', # Оставить
#             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', # Оставить
#             'sec-fetch-site': 'same-origin', # Оставить
#             'sec-fetch-mode': 'cors', # Оставить
#             'sec-fetch-dest': 'empty', # Оставить
#             'referer': referer,
#             'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5', # Оставить
#             'cookie': 'tt_webid=6926967686699697670; tt_webid_v2=6926967686699697670; ttwid=1%7C-Pf22eSHkgheOiac4U_hfV9E40k3svSTiyUv1fZGyOM%7C1613134207%7Cd8303836d7c5fe59e24af07560fff61efb5176eb152c1e5120c1b920d4eff7b4; passport_csrf_token=3e864d33e1b1774d956c4fc3131de32b; passport_csrf_token_default=3e864d33e1b1774d956c4fc3131de32b; uid_tt=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; uid_tt_ss=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; sid_tt=9075b4eb80838c9247d7933b0a4d4133; sessionid=9075b4eb80838c9247d7933b0a4d4133; sessionid_ss=9075b4eb80838c9247d7933b0a4d4133; store-idc=alisg; store-country-code=ru; sid_guard=9075b4eb80838c9247d7933b0a4d4133%7C1613134227%7C5184000%7CTue%2C+13-Apr-2021+12%3A50%3A27+GMT; odin_tt=574552fa1292cc681daa70c18beef58130563e7cfff3396d9e65f1645a352224814648d17b65e0350f9030f3bd9e4c8790d3ba34ca24697a412f163ef07244fb6f590d6dcc54c6b0efdcca39603366ea; MONITOR_WEB_ID=6926967686699697670; _ga=GA1.2.896708646.1615818649; s_v_web_id=verify_kmxgwki8_Fxz90IPP_XZwi_4XQs_99TG_AfLnxuB6PQj0; tt_csrf_token=x5ogawzb_QDPU8URJixH9b9s; bm_sz=ADB21068722BD627EE1EE0C38FDE98EC~YAAQBiWKUnzY4ap4AQAAQlBTwQvLd+upuHN6BecO9Ng+2Imp5BDqct4ifR2hG/17+NZ4mzN4eYCJVs/UYeS+iPSV2AnrSrdAXrCahVuI3LSEWJDf+hVrA9MMh2YPOZ+nTMRsr/TVHgXNj9JmbVnQZETDu/mWv9+Nqhcfvp6p9cp+P8v3j/M2GAGEezno2DVI; _abck=04D2CF35FC8971153BBDA9F8BF46734C~0~YAAQBiWKUn3Y4ap4AQAAQlBTwQUCSO8USBibWIA50lm98daebhMkM0Czsa/oyQhpxM5k0hVQVRSBfxMxtOvWmw/PYMv+nPXY9I7/3fD8wvNqG52tkz22PEr1p7ELH/gVOfAVKcMpv8AxW/8FA5Rm/sWTqAeFQypRkIkPlOgb26ccymaXPEO28CcWK1iM5YOP/Nf923aVg57k0Z003LU1U3GBxH04Qn05nhzshcx54qsKMfoYGVtzKfg+yTHSaklcy601dzvkLCVAZTBCiLmu/RnlR0f0j/DHqm1jM+qPM2SA1c2Gfc5IgcV55893QeRQlatOEg4OHK3KXSUsuka7bmCfyydXlE5c1E2IbRywntVcoTazcBOSsfSaKgBUhDqnwmnQL3M=~-1~-1~-1; R6kq3TV7=AOFUU8F4AQAAySETBvrdffUN6gvMYn6iXHxTc7QGpNA7mJDIGXXSEY5BL8Hx|1|0|6034076e35c2191645a706fdbab06d8c94a47276; csrf_session_id=beac3f98992342f0a704801619be4c79; bm_mi=D6B551ED62BF8FEB4D2397E7BBEF155C~ssNoNitiYE4k2OFhdM9JcWCLFzXA9dKWBoxZKiaho9ZHc7qt+sz82fLjPWV1DiP8ZDxBncCGjyaT7BBgkbq/BnAV9MyhxCdTrk2WPgfqKu4DM/cPzR4ZoLJyi6lxwkOQBEe6ihKWc1wj+WeHQjkDjRo/x0DStYdQcScb7bITPWRSx8u/I2+CSCzNWbN+GkWKbB1XI3BpaXpbWn6T47ul+jJ9M8Y95ph/DTUkHRWkZQsg1nxZ7DDDPPfVsBteevd2wfeWxj+YftNy81nc997+1w==; bm_sv=1187DCE3972FB6AAC3BA8E427A45891E~GD7NUSYugXnQ3Y5Ck76jPTgn0651fjW/MfXkXv0bABLVIpWdoYoyJbOmA2vmhgi1q/wZzY8Y2wG9IndXLG2VNXkoxKbeVTUT4IybUymp008iV7KTm9ON42lVaTxUKTobb9ycT23YipIPNN+R8uzP7+IB6sMBRJyrALFMCtCj9J4=; ak_bmsc=F948465D81B19B53CE234CBFDB83A9D9528A25055D6A00001E2373601AC2415E~plEbI7L4oaCskK+WFLJZdmdH4thRirF7TVIS8o9MOeT+ioNMU3Yz/3LeabVmNJVotPJ2noOUEmIgjILAviVqzxXGu/a/ycU4JMRfsBWdANxpYbZ7S59tRWVgcl88IArXDFEl21nZkDZwQ41wUuO5dH1Xmw1k3tVInxe436NZxR+RtiV1/Z0dmcULkd4M1bIU2xw7oQWrinn61BRQybZozsC+zwbFFqEeOx0IV7c4Cd3kDxVl2UwiUN2AC7P9FJd/D5uq9hSuvCeZ5XAUaOAXaEz4N94Sy9QHOFgScBM5qlI5BQOY/Y5Z/hnM5klTrEq1ui; cmpl_token=AgQQAPP2F-RMpY-gSj0Gth08-kARWr_Hf4R9YPgcAQ; _abck=04D2CF35FC8971153BBDA9F8BF46734C~-1~YAAQTgo9FyEuor54AQAAH/jcwQUlEZsiD90RHHMoen+BuX9e7nyk+rkYpf0/55FRzRJFyVTmX0M8Zcq9azTNBIUdyxuskB3b1245lyg0Om6+CoECQ8HSZdk3B6BNVb9U+3i8KIV34YtzU3HW4HC1Yg45JqHkWk5gN+7yO1eqC7+6OrWm5dezGxVhVr9e3JgzFDJfihKZZNLJmcH7Mv8z3l2CQNapWNdFjmhNlojpinjHcc9Xdpp8q9QKaI+UyoQfUIX079mYGWpWJC6bmibbyCxbEl9nadukZRMAOKDua3v8LTaHH6xJ+Dp3drl86kSWVQzif6SRf9te93wm3stNDydTuMF9XSoGWJbWwI3BOTKysVDPc65wMoYn/jWgT8hS/JuOOi4=~0~-1~-1'
#         }
#         # ОБНОВЛЯТЬ КУКИ
        

#         response = requests.request("GET", url, headers=headers, data=payload)
        #url = f"https://www.tiktok.com/api/comment/list/?aid=1988&app_name=tiktok_web&device_platform=web_pc&referer=&root_referer=&user_agent=Mozilla%2F5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&cookie_enabled=true&screen_width=1280&screen_height=1024&browser_language=ru-RU&browser_platform=Win32&browser_name=Mozilla&browser_version=5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&browser_online=true&ac=4g&timezone_name=Europe%2FMoscow&page_referer=https:%2F%2Fwww.tiktok.com%2F@russiagreecedata%3Flang%3Dru-RU&priority_region=RU&verifyFp=verify_kmxgwki8_Fxz90IPP_XZwi_4XQs_99TG_AfLnxuB6PQj0&appId=1233&region=RU&appType=m&isAndroid=false&isMobile=false&isIOS=false&OS=windows&did=6926967686699697670&tt-web-region=RU&uid=6890125463646422021&insert_ids=&aweme_id={aweme_id}&cursor={cursor}&count={count}&app_language=ru&current_region=RU&fromWeb=1&channel_id=0&_signature=_02B4Z6wo00901BMNcDQAAIDCz0kNRKmHE3ATDXSAAGS2dd"
        url = f"https://www.tiktok.com/api/comment/list/?aid=1988&app_name=tiktok_web&device_platform=web_pc&referer=&root_referer=&user_agent=Mozilla%2F5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&cookie_enabled=true&screen_width=1280&screen_height=1024&browser_language=ru-RU&browser_platform=Win32&browser_name=Mozilla&browser_version=5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F89.0.4389.114+Safari%2F537.36&browser_online=true&ac=4g&timezone_name=Europe%2FMoscow&priority_region=RU&verifyFp=verify_kmxgwki8_Fxz90IPP_XZwi_4XQs_99TG_AfLnxuB6PQj0&appId=1233&region=RU&appType=m&isAndroid=false&isMobile=false&isIOS=false&OS=windows&did=6926967686699697670&tt-web-region=RU&uid=6890125463646422021&insert_ids=&aweme_id={aweme_id}&cursor={cursor}&count={count}&app_language=ru&current_region=RU&fromWeb=1&channel_id=0&_signature=_02B4Z6wo00901BMNcDQAAIDCz0kNRKmHE3ATDXSAAGS2dd"

        payload={}
        headers = {
          'authority': 'www.tiktok.com',
          'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
          'accept': 'application/json, text/plain, */*',
          'sec-ch-ua-mobile': '?0',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
          'sec-fetch-site': 'same-origin',
          'sec-fetch-mode': 'cors',
          'sec-fetch-dest': 'empty',
          'referer': f'https://www.tiktok.com/@torryhermann/video/{aweme_id}?is_copy_url=1&is_from_webapp=v1',
          'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5',
          'cookie': 'tt_webid=6926967686699697670; tt_webid_v2=6926967686699697670; ttwid=1%7C-Pf22eSHkgheOiac4U_hfV9E40k3svSTiyUv1fZGyOM%7C1613134207%7Cd8303836d7c5fe59e24af07560fff61efb5176eb152c1e5120c1b920d4eff7b4; passport_csrf_token=3e864d33e1b1774d956c4fc3131de32b; passport_csrf_token_default=3e864d33e1b1774d956c4fc3131de32b; uid_tt=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; uid_tt_ss=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; sid_tt=9075b4eb80838c9247d7933b0a4d4133; sessionid=9075b4eb80838c9247d7933b0a4d4133; sessionid_ss=9075b4eb80838c9247d7933b0a4d4133; store-idc=alisg; store-country-code=ru; sid_guard=9075b4eb80838c9247d7933b0a4d4133%7C1613134227%7C5184000%7CTue%2C+13-Apr-2021+12%3A50%3A27+GMT; odin_tt=574552fa1292cc681daa70c18beef58130563e7cfff3396d9e65f1645a352224814648d17b65e0350f9030f3bd9e4c8790d3ba34ca24697a412f163ef07244fb6f590d6dcc54c6b0efdcca39603366ea; MONITOR_WEB_ID=6926967686699697670; _ga=GA1.2.896708646.1615818649; s_v_web_id=verify_kmxgwki8_Fxz90IPP_XZwi_4XQs_99TG_AfLnxuB6PQj0; tt_csrf_token=x5ogawzb_QDPU8URJixH9b9s; R6kq3TV7=AOFUU8F4AQAAySETBvrdffUN6gvMYn6iXHxTc7QGpNA7mJDIGXXSEY5BL8Hx|1|0|6034076e35c2191645a706fdbab06d8c94a47276; csrf_session_id=beac3f98992342f0a704801619be4c79; cmpl_token=AgQQAPP2F-RMpY-gSj0Gth08-kARWr_Hf4R9YPgdmA; ak_bmsc=368058937B79C7892D4DBD6CC89AC0AC528A25064D69000056457360D43C9F2B~pl0Fr2xXrkku2YXnVEk4ux5DAZSTelS0Osuue1JJplM36DnLhFCKvEEe3QQbDHPdJMuCKAljoEauMtBXGZcN2HeGXN2bc2zjQv4q0H6t7USIc9t6+ctj3CGNdeH7gRr60dllN5oLsFgjMkbFdfW1rh0txesB5zDJwQAVzghKf1pUSC38eGy0J43FyTFP9OgxMwMRLLCtBguTWRBiOAW4zYQVrpSJKWxlGvWv04JlqrVrw=; bm_sz=EDA3400A71179526978EF0F92F08F815~YAAQBiWKUkDk4ap4AQAA4dpGwgtI58QKsqY8/q18duFc0HUMDYRSxh8cKr9CHfHSiGuIVLjC4eXgCMeZUETXJcTlaFdayuJaky6ZCWyh2b5RgWl6gVfTXT2iONXpqdjXvfd9CBZTIYjW3ZN1c+LCGnyoZVoBK9LuSea77Ja3GS9TicVIjaHwCLA870B8lXFl; _abck=04D2CF35FC8971153BBDA9F8BF46734C~0~YAAQBiWKUkHk4ap4AQAA4dpGwgWrck/+ikbp/ehtYZUeeH23GSF+W2a5tsb3WJfa7MoBM4ozxoW9BOHUgNh/E2dsO/rKGNzWvweGyx6Amycz7jiGy+Y556QzailrBRd+3q0gZ2lYt1Lt071nqe7NrFM3nreY0WRN2ISkk9OkO1XZJ0EiYAXyB6P4vULDOGGJOmOlGrds844XfoWv6iySYmnr6D91r3UOGMJLz3zAfURfFMYHXaK3mCCZv7ELlWFowpPBTFjIHeFwGdeBK4yDCV/KlsknlckFRfJ/bwZSPdQTxf8sX47RNsFDf7Ejg6D143FizWD2NjG4kSycIl0FTLEjSVnns3P5ccH4j9cACcTf0aj7nZCqzOQ9ousVjd0xlCmP8Tg=~-1~-1~-1; bm_sv=03B75C5F4A54EE9AC1BC7254B975104A~NAeI1AwynosACdbeChQSiOWGjyUqneR+5Ohqi8QXAYMUgczVmCXkqyRtyiSLq5tCRUmh/tp0WfVRkXPJxeqyx5dyrIv0UnUVfgRucVsveeq5OR5rxEvfmWv+O4fc24kHXLsx+A+UazQxB/Iarf1RezVleFxN4zzmNjpHCEso9bI=; _abck=04D2CF35FC8971153BBDA9F8BF46734C~-1~YAAQuMbPF2OsmHJ4AQAADkZSwgVjDq8P+T7yD70j/zaJqSOwF9qjlVq540Q4YcT5LTgeazCRwG+AahZX0SI7Yuj3J9V28Xr5+HeIugvD3xfU5xI+FaGeVcv4YA2qJQ6K3sHe6wqwJZPslk9SywLvqXpnPOUw3riBdqTjul2ZNlxemLe4hT9Cjs5PZmm7I7ucYGj8GPQDbq2uVxDpk1a5M/25tNZFNmEh5NhEB3U8kzd1PiuIK+7aJfEh5VrukE5DioEswm40vX2YMAPCr9bTkhZaRlkkrwJRGWplUj3U/+eIbtELir3K4xkXrOOgQ8wWcU3jg7SF1lvXtyb30Rzgiew071wkP+BjOPnN8JKwkrf6oWwisX/imeefvFgzPyNySFqoad0=~0~-1~-1'
        }
        
        response = requests.request("GET", url, headers=headers, data=payload)
        
        print (response.text)

        time.sleep(random.uniform(2, 6))

        try:
            if len (response.json()['comments']) < 50:
                for i in response.json()['comments']:
                    all_responses [ids] = i
                    ids = ids + 1
                print (len (response.json()['comments']))
                print ('Это конец')
                break
            else:
                for i in response.json()['comments']:
                    all_responses [ids] = i
                    ids = ids + 1
                count = count + offset
                cursor = cursor + offset
                print (len (response.json()['comments']))
                print ('Это не конец')
                continue
        
        except TypeError:   # Код уходит на эту ошибку, если число комментариев — ровно 50
                            # Он делает новую итерацию и получает пустоту
                            # Поэтому мы возвращаем cursor и count к прежним значениям
                
            print ('НЕ ПО ПЛАНУ')
            count = count - offset
            cursor = cursor - offset
            url = f"https://www.tiktok.com/api/comment/list/?aid=1988&app_name=tiktok_web&device_platform=web_pc&referer=&root_referer=&user_agent=Mozilla%2F5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F87.0.4280.141+Safari%2F537.36&cookie_enabled=true&screen_width=1280&screen_height=1024&browser_language=ru-RU&browser_platform=Win32&browser_name=Mozilla&browser_version=5.0+(Windows+NT+10.0%3B+Win64%3B+x64)+AppleWebKit%2F537.36+(KHTML,+like+Gecko)+Chrome%2F87.0.4280.141+Safari%2F537.36&browser_online=true&ac=4g&timezone_name=Europe%2FMoscow&page_referer=https:%2F%2Fwww.tiktok.com%2F@russiagreecedata%3Flang%3Dru-RU&priority_region=RU&verifyFp=verify_kk14bgxh_Ey7Ty0Nw_0w04_4iQX_8eBP_rzF6PEPNP7NM&appId=1233&region=RU&appType=m&isAndroid=false&isMobile=false&isIOS=false&OS=windows&did=6916212750071776773&tt-web-region=RU&uid=6890125463646422021&insert_ids=&aweme_id={aweme_id}&cursor={cursor}&count={count}&app_language=ru&current_region=RU&fromWeb=1&channel_id=0&_signature=_02B4Z6wo00901zALRiwAAIDB7E87Xdu9mQMwCkKAAKwR47"
            response = requests.request("GET", url, headers=headers, data=payload)
            for i in response.json()['comments']:
                all_responses [ids] = i
                ids = ids + 1
            print (len (response.json()['comments']))
            print ('Это конец')
            break
        
        except KeyError:
            try:
                if response.json()['status_msg'] == 'Вы вышли из аккаунта':
                    print ('Кажется, Вы вышли из аккаунта, поменяйте входные данные — cookie и sec-ch-ua в headers')
                else:
                    print (response.json()['status_msg'])
            except:
                raise KeyError
            print ('Код прекращает работу')
            break
            
    all_responses_copy = all_responses.copy()

    for i in all_responses.values():
        if i ['reply_comment'] is not None:
            for k in i['reply_comment']:
                all_responses_copy [ids] = k
                ids = ids + 1
                if k['reply_comment'] is not None:
                    print ('O wow!')
                    for z in k['reply_comment']:
                        all_responses_copy [ids] = z
                        ids = ids + 1
    
    response_df = pd.DataFrame.from_dict (all_responses_copy).T
    return all_responses_copy, response_df


def quick_check (all_links_to_parse):
    
    ''' Функция принимает на вход список ссылок на TikTok-видео, проверяет их валидность 
    (продолжает ли видео находиться в открытом доступе, или же оно скрыто/ удалено) и возвращает список проверенных ссылок. '''

    for referer in tqdm (all_links_to_parse):
        
        payload={}
        headers = {
          'authority': 'www.tiktok.com',
          'x-kl-ajax-request': 'Ajax_Request',
          'accept': 'application/json, text/plain, */*',
          'sec-ch-ua-mobile': '?0',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
          'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
          'sec-fetch-site': 'same-origin',
          'sec-fetch-mode': 'cors',
          'sec-fetch-dest': 'empty',
          'referer': referer,
          'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5',
          'cookie': 'tt_webid_v2=6916212750071776773; tt_webid=6916212750071776773; ttwid=1%7CcdWyZ48EUgYhNscoWjjs4-x_r5maVNmW9iawlop2cNU%7C1610798758%7C6f5d0da3f04946eeaf6ced45118b69fd8ae2fea758073e919b5f6a01e8285087; passport_csrf_token=e5df83bd420baf05772ded41579ebfad; passport_csrf_token_default=e5df83bd420baf05772ded41579ebfad; store-idc=alisg; store-country-code=ru; cmpl_token=AgQQAPP2F-RO0o-gSj0Gth08xZrL6ZCRf4d9YPo--g; sid_guard=e6c07d4a41d789c3a181163cba66673e%7C1610804042%7C5184000%7CWed%2C+17-Mar-2021+13%3A34%3A02+GMT; uid_tt=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; uid_tt_ss=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; sid_tt=e6c07d4a41d789c3a181163cba66673e; sessionid=e6c07d4a41d789c3a181163cba66673e; sessionid_ss=e6c07d4a41d789c3a181163cba66673e; MONITOR_WEB_ID=6916212750071776773; tt_csrf_token=GPy9U4CN50q1WNOtsFtAXWu-; s_v_web_id=verify_kk14bgxh_Ey7Ty0Nw_0w04_4iQX_8eBP_rzF6PEPNP7NM; csrf_session_id=a0f71447990040f7a19b35e99c980613; odin_tt=ba60c4b6907072ee26ad5b2462547eea1e16da5e658698dae644227ead9b99850d405faf8246f2d1e9087a40287e9af988a24c4ce98107ebed7edbbf8a3909636884ee832bd5e3252181f720a8b0512b'
        }
        
        try:
            response = requests.request("GET", referer, headers=headers, data=payload)
            if response.status_code == 404:
                print ('Видео по ссылке больше не доступно, и мы удаляем его из выборки')
                all_links_to_parse.remove (referer)
                time.sleep(random.uniform(0.5, 2))
            elif response.status_code == 200:
                time.sleep(random.uniform(0.5, 2))
                pass
        except:
            print ('Некорректная ссылка, пропускаем')
            pass
            
    return all_links_to_parse

def unique_characteristics (referer):
    
    ''' Функция позволяет собирать уникальную информацию о видео, как то:
    1. уникальный идентификатор видео (aweme_id)
    2. подпись к видео (description)
    3. время публикации в формате timestamp (timestamp_create_time)
    4. время публикации в человекочитаемом формате (type = string) (date)
    5. id автора видео (author_id)
    6. имя автора видео (author_name)
    7. никнейм автора (author_nickname)
    8. музыку в видео (music)
    9. лайки (likes)
    10. шеры (shares)
    11. комментарии (comments)
    12. просмотры (views)
    13. надписи на стикерах в видео (если есть) (sticker_text)
    14. ссылка на оригинальное видео, на которое был сделан ститч или дуэт (stitched_original_link).
    
    Функция возвращает список этих элементов. 
    В списке на втором месте, после aweme_id, ссылка на это видео. '''
    
    aweme_id = get_aweme_id_final (referer) # id видео: взять из URL после video/, но до ?

    payload={}
    headers = {
      'authority': 'www.tiktok.com',
      'x-kl-ajax-request': 'Ajax_Request',
      'accept': 'application/json, text/plain, */*',
      'sec-ch-ua-mobile': '?0',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
      'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
      'sec-fetch-site': 'same-origin',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': referer,
      'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5',
      'cookie': 'tt_webid_v2=6916212750071776773; tt_webid=6916212750071776773; ttwid=1%7CcdWyZ48EUgYhNscoWjjs4-x_r5maVNmW9iawlop2cNU%7C1610798758%7C6f5d0da3f04946eeaf6ced45118b69fd8ae2fea758073e919b5f6a01e8285087; passport_csrf_token=e5df83bd420baf05772ded41579ebfad; passport_csrf_token_default=e5df83bd420baf05772ded41579ebfad; store-idc=alisg; store-country-code=ru; cmpl_token=AgQQAPP2F-RO0o-gSj0Gth08xZrL6ZCRf4d9YPo--g; sid_guard=e6c07d4a41d789c3a181163cba66673e%7C1610804042%7C5184000%7CWed%2C+17-Mar-2021+13%3A34%3A02+GMT; uid_tt=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; uid_tt_ss=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; sid_tt=e6c07d4a41d789c3a181163cba66673e; sessionid=e6c07d4a41d789c3a181163cba66673e; sessionid_ss=e6c07d4a41d789c3a181163cba66673e; MONITOR_WEB_ID=6916212750071776773; tt_csrf_token=GPy9U4CN50q1WNOtsFtAXWu-; s_v_web_id=verify_kk14bgxh_Ey7Ty0Nw_0w04_4iQX_8eBP_rzF6PEPNP7NM; csrf_session_id=a0f71447990040f7a19b35e99c980613; odin_tt=ba60c4b6907072ee26ad5b2462547eea1e16da5e658698dae644227ead9b99850d405faf8246f2d1e9087a40287e9af988a24c4ce98107ebed7edbbf8a3909636884ee832bd5e3252181f720a8b0512b'
    }

    response = requests.request("GET", referer, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'lxml') # загружаем DOM ссылки
    txt = soup.find_all('script', {'type':'application/json'})[0].text

    for_info = json.loads(txt)['props']['pageProps']['itemInfo']['itemStruct']
    aweme_id = for_info ['id']
    description = for_info ['desc']
    timestamp_create_time = for_info ['createTime']
    ts = time.gmtime(timestamp_create_time)
    date = time.strftime("%d.%m.%Y", ts)
    author_id = for_info ['author']['id']
    author_name = for_info ['author']['nickname']
    author_nickname = for_info ['author']['uniqueId']
    music = for_info ['music']['title']
    likes = for_info ['stats']['diggCount']
    shares = for_info ['stats']['shareCount']
    comments = for_info ['stats']['commentCount']
    views = for_info ['stats']['playCount']
    try:
        raw_sticker_text = for_info ['stickersOnItem'][0]['stickerText']
        sticker_text = ''
        for i in raw_sticker_text:
             sticker_text = ' '.join ((sticker_text, i))
    except IndexError:
        sticker_text = ''
    stitched_original_aweme_id = for_info ['textExtra'][0]['awemeId']
    stitched_original_creator = for_info ['textExtra'][0]['userUniqueId']
    stitched_original_link = ''
    if (stitched_original_aweme_id != '') & (stitched_original_creator != ''):
        stitched_original_link = 'https://www.tiktok.com/@' + stitched_original_creator + '/video/' + stitched_original_aweme_id
    row = [aweme_id, referer, description, timestamp_create_time, date, author_id, author_name, author_nickname, music,
          likes, shares, comments, views, sticker_text, stitched_original_link]
    return row



def change_line_break (text):
    
    ''' Функция принимает на вход текст и заменяет в нём переносы строк на пробелы.  '''
    
    if '\n' in text: 
        text = text.replace ('\n', ' ')
    return text



def drop_hashtag (text):
    
    ''' Функция принимает на вход текст и удаляет в нём символ хэштега (#).  '''
    
    if '#' in text: 
        text = text.replace ('#', '')
    return text



def user_minimum_from_dicts (user_data):
    
    ''' Функция сокращает данные пользователя, оставившего комментарий, до имени, ника и sec_uid. 
    Принимает на вход словарь ({}) и возвращает словарь с тремя ключами'''
    
    new_dict = {}
    new_dict ['nickname'] = user_data ['nickname']
    new_dict ['unique_id'] = user_data ['unique_id']
    new_dict ['sec_uid'] = user_data ['sec_uid']
    return new_dict



def clear_comments_table_user_dict_without_saving (comments_table_to_upd):
    
    ''' Функция принимает на вход грязный пандас датафрейм с комментариями (comments_table_to_upd).
    Возвращает чистый и аккуратный датафрейм.
    В отличие от функции clear_comments_table, которая работает с датасетами, загруженными из csv-файла,
    эта функция работает с датасетами, которые получены прямо в ходе парсинга,
    то есть с теми, у которых в столбце user словари, а не строки.'''
    
    comments_table_to_upd ['text'] = comments_table_to_upd ['text'].apply (change_line_break)
#     comments_table_to_be_clear = comments_table_to_upd.drop (['comment_type', 'reply_to_reply_id', 'status', 
#                                                               'stick_position', 'text_extra', 'user_buried', 'user_digged'], 
#                                                              axis = 1)
    columns_list_for_drop = ['comment_type', 'reply_to_reply_id', 'status', 'stick_position', 'text_extra', 
                         'user_buried', 'user_digged']
    for every_column in columns_list_for_drop:
        if every_column in comments_table_to_upd.columns:
            comments_table_to_be_clear = comments_table_to_upd.drop (every_column, axis = 1)
        else:
            pass
    comments_table_to_be_clear ['user'] = comments_table_to_be_clear ['user'].apply (user_minimum_from_dicts)
    comments_table_to_be_clear ['user_nickname'] = [d.get('nickname') for d in comments_table_to_be_clear ['user']]
    comments_table_to_be_clear ['user_unique_id'] = [d.get('unique_id') for d in comments_table_to_be_clear ['user']]
    comments_table_to_be_clear ['user_sec_uid'] = [d.get('sec_uid') for d in comments_table_to_be_clear ['user']]
    ts = [time.gmtime(i) for i in comments_table_to_be_clear ['create_time']]
    comments_table_to_be_clear ['date'] = [time.strftime("%d.%m.%Y", i) for i in ts]
    return comments_table_to_be_clear



def hashtags (text):
    
    ''' Принимает на вход текст и возвращает список хэштегов из него. '''
    
    list_of_hashtags = re.findall(r'#\w+', text)
    if '#stitch' in list_of_hashtags:
        list_of_hashtags.remove('#stitch')
    return list_of_hashtags



def stitches_parsing_workflow ():
    
    ''' Функция просит пользователя ввести ник TikTok-аккаунта, автора видео, ститчи по которому нужно собрать. 
    Функция ничего не возвращает, но:
    1. Создаёт папку по имени ника пользователя (автора оригинального видео)
    2. Сохраняет в папку все ссылки на заститченные видео, которые находятся через Google-поиск, как csv-файл
    3. Сохраняет в папку уникальные характеристики этих видео как csv-файл 
    4. Сохраняет в папку датасет с комментариями к этим видео как csv-файл
    5. Сохраняет в папку список релевантных тэгов из результатов Google-поиска и из подписей к видео как csv-файл 
    с указанием источника, откуда получен тэг (Google/ caption/ caption_internal_links)
    6. Сохраняет в папку тот же список без указания источника, откуда получен тэг, 
    как gexf-файл для удобного преобразования в граф
    
    В целом функция выполняется около 15 минут.
    Код издаёт звук, когда завершает работу. '''
    
    nickname_stitch_needed = input ('Введите ник пользователя, для видео которого нужны ститчи: ')
    
    try:
        path = os.path.join (os.getcwd(), nickname_stitch_needed)
        os.mkdir (path)
    except OSError:
        print ('Creation of the directory %s failed' % path)
    else:
        print ('Successfully created the directory %s' % path)
    print (f'Папка {nickname_stitch_needed} успешно создана в текущей директории')
    
    print ('Начинаем сбор ссылок на видео и тэги из Google-поиска')
    stitch_treasury_result = is_it_a_video (stitch_treasury (nickname_stitch_needed))
    print ('Собрали ссылки на видео и тэги из Google-поиска')
    
    print ('Выделяем из результатов тэги')
    tags_nickname = unique_tags_only (stitch_treasury_result [0])
    with open (os.path.join (path, f'{nickname_stitch_needed}_tags.csv'), 'w', newline = '') as tags:
        writer = csv.writer (tags, delimiter = ",")
        writer.writerow (['original_hashtag', 'relative_hashtag', 'source']) 
        for i in tags_nickname:
            print (i)
            writer.writerow ([f'stitch with @{nickname_stitch_needed}', i, 'Google'])
    print ('Записали первую часть тэгов в файл')
    
    print ('Выделяем все ссылки на видео для парсинга')
    links_nickname = stitch_treasury_result [1]
    all_links_to_parse = unique_list_of_links_nickname (links_nickname)
    all_links_to_parse = quick_check (all_links_to_parse)
    with open (os.path.join (path, f'{nickname_stitch_needed}_links.csv'), 'w', newline = '') as links:
        writer = csv.writer (links, delimiter = ",")
        for i in all_links_to_parse:
            writer.writerow (i.split())
    print ('Сохранили все ссылки в файл')

    print ('Приступаем к парсингу уникальных характеристик видео')
    print ('Не переживайте, процесс может занять некоторое время')
    row_titles = ['aweme_id', 'link', 'description', 'timestamp_create_time', 'date', 'author_id', 'author_name', 
                  'author_nickname', 'music', 'likes', 'shares', 'comments', 'views', 'sticker_text', 'stitched_original_link']
    unique_char_table = pd.DataFrame (columns = row_titles)
    for i in tqdm (all_links_to_parse, position=0, leave=True):
        un_char = unique_characteristics (i)
        unique_char_table.loc [len (unique_char_table)] = un_char # Добавляем ряд к датафрейму
        time.sleep (random.uniform(2, 6))
    unique_char_table ['hashtags'] = unique_char_table ['description'].apply (hashtags)
    unique_char_table.to_csv (os.path.join (path, f'{nickname_stitch_needed}_unique_characteristics.csv'))
    print ('Записали уникальные характеристики видео в файл')
    
#     print ('Приступаем к парсингу комментариев для видео')
#     print ('Не переживайте, процесс может занять некоторое время')
#     print ('Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,') 
#     print ('сколько примерно комментариев под видео, которое парсится на данный момент.')
#     print ('То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев')
    
#     comments_table = pd.DataFrame ()
#     for i in tqdm (all_links_to_parse, position=0, leave=True):
#         video_comments = comments_parse_upd (i)
#         comments_table = comments_table.append (video_comments[1], ignore_index = True)
#     comments_table_to_save = clear_comments_table_user_dict_without_saving (comments_table)
#     comments_table_to_save.to_csv (os.path.join (path, f'{nickname_stitch_needed}_comments_table.csv'), sep = ';')
#     print ('Почистили таблицу комментариев и записали в файл')
    
    print ('Приступаем к сбору тэгов из подписей к видео')
    with open (os.path.join (path, f'{nickname_stitch_needed}_tags.csv'), 'a', newline = '', encoding = 'utf-8') as tags:
        writer = csv.writer (tags, delimiter = ",")
        for i in unique_char_table ['hashtags']:
            for j in i:
                print (j)
                writer.writerow ([f'stitch with @{nickname_stitch_needed}', drop_hashtag (j), 'caption'])
            for a in range (len (i)):
                for b in range (a + 1, len (i)):
                    writer.writerow ([drop_hashtag (i[a]), drop_hashtag (i[b]), 'caption_internal_links'])
    print ('Записали вторую часть тэгов в файл')
    
    print ('Превращаем csv-файл тэгов в gexf-файл для создания графа')
    tags_to_gexf = pd.read_csv (os.path.join (path, f'{nickname_stitch_needed}_tags.csv'), encoding = 'utf-8')
    tags_to_gexf = tags_to_gexf.drop ('source', axis = 1)
    tags_to_gexf.columns = ['source', 'target']
    gexf_graph = nx.from_pandas_edgelist (tags_to_gexf, create_using = nx.MultiGraph ())
    nx.write_gexf (gexf_graph, os.path.join (path, f'{nickname_stitch_needed}_graph.gexf'))
    print ('Gexf-файл успешно сохранён')
    
    winsound.Beep (440, 1000)

    print ('Ура! Код завершил работу. Проверьте Ваши файлы')
    
    
    

def video_response_for_downloading (url):
    
    ''' Функция принимает на вход URL TikTok-видео и возвращает контент для последующей записи в видео-файл. '''
    
    headers = {
      'Connection': 'keep-alive',
      'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
      'sec-ch-ua-mobile': '?0',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
      'Accept': '*/*',
      'Sec-Fetch-Site': 'same-site',
      'Sec-Fetch-Mode': 'no-cors',
      'Sec-Fetch-Dest': 'video',
      'Referer': 'https://www.tiktok.com/',
      'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5',
      'Cookie': 'tt_webid=6926967686699697670; tt_webid_v2=6926967686699697670; ttwid=1%7C-Pf22eSHkgheOiac4U_hfV9E40k3svSTiyUv1fZGyOM%7C1613134207%7Cd8303836d7c5fe59e24af07560fff61efb5176eb152c1e5120c1b920d4eff7b4; passport_csrf_token=3e864d33e1b1774d956c4fc3131de32b; passport_csrf_token_default=3e864d33e1b1774d956c4fc3131de32b; uid_tt=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; uid_tt_ss=4d5af9958f703f8488679a8462dea4362836c8d4f88a6f2a69440c2f9324505b; sid_tt=9075b4eb80838c9247d7933b0a4d4133; sessionid=9075b4eb80838c9247d7933b0a4d4133; sessionid_ss=9075b4eb80838c9247d7933b0a4d4133; store-idc=alisg; store-country-code=ru; sid_guard=9075b4eb80838c9247d7933b0a4d4133%7C1613134227%7C5184000%7CTue%2C+13-Apr-2021+12%3A50%3A27+GMT; odin_tt=574552fa1292cc681daa70c18beef58130563e7cfff3396d9e65f1645a352224814648d17b65e0350f9030f3bd9e4c8790d3ba34ca24697a412f163ef07244fb6f590d6dcc54c6b0efdcca39603366ea; _ga=GA1.2.896708646.1615818649; cmpl_token=AgQQAPP2F-RMpY-gSj0Gth08-kARWr_Hf4R9YPjrJQ; tt_csrf_token=s5rU_UH_rOzyY0gbm707bWmt; bm_sz=0B4501FDA3E67593229993870522EE16~YAAQByWKUmok+6t4AQAACA8psgtheMIlWTcewismhRxyJ1r0q6LghUXdICCHrVWDQeaheH2HB0/i+kF6VMVgKJdreeUoc70ECUiash9frKht+u31nMH4gt4KADhtVmER/rMGQ7lxxehLmKhMlRJYWbrPRw+53WPHIVJp3z3zjkHoLtSdyhBmuTX5eiGou+y0; _abck=04D2CF35FC8971153BBDA9F8BF46734C~0~YAAQByWKUmsk+6t4AQAACA8psgVadAmBdjj7aB5vyrNwQFwzMieEl+m4TTES0J13H81h5sUgg2R/RlkiR1J4CIgdxKCljMtpF1muaQnVPYSH0AZ8qtLQYfJO4SfyUoN7iRGNgSQjxDRJftt00CXR9XqtJyawHYbXjwInzXBspXrpgF+2Mpjk8/LNe7q4siw3lTQ7Ht7+R+FUyb7fpbDlLGmhYN4LJPJV3A8/Rxr2NauItyqZDKrl6HiJEsKtrXDMhk+QlSBnIscbsh8h9uKmUcxPz59WyJQbTr0IUoOyzIZF1+JnfmtopTkiu050Hz7ycTBYBEX720pUibllv/2CnEnAT4UYRT0jzL2dZ4WOJoGZN1xeCdQsRTMF4YngDbpW8lAuBlI=~-1~-1~-1; ak_bmsc=2AE5712070BE4F8A8CA7343636ACE60B528A2507E531000022256F609F3C5B28~plQsuttkfFHSQ8S+SszK9RFtFRyfz6eETrzTeByhjTlzkpB6p7T+yBcLGpV5RUePTAqJsBo9jnQMA1Y5eY1bKy9dPz2Q797LEIfsg0UoCCyEnVs3WvnjTtGUkO18Bf51HxbMF4Aiz6L6I1Z61Ey/C5k/x6qnnhmXPfCsh7YfiL1CqifbYwbJ1CrT5iJb2GuObOPGer/B9E45q42eCImBRoQWjU4qoXuvg9G6qy4pid2IYpdxgIx2fHWRpK1Sel8XYlhgp4ubP1Eqji75ht2IL3F7ptHKVcoGT7oCY4NaVIedNa6N9feRpEPLMOQNzDxAI+gATp0oho88gqMAzbRF2kMg==; R6kq3TV7=AFYVKbJ4AQAAyB-thZ-OoTOiby3CDIdfsCCqGk4RqfLPlwNr2WcJSwa6tSXa|1|0|2cfb4310802e69f6825fe224f7d9a2f638b9d9d2; bm_mi=914D9AEAA597FF08851DE1907CAF9689~SWCFtjvzN1rLWZ4iKRMGyXboNrJJTfeU2s4xOvAIWIhbO8Q/uI1wujooW2nyLdlEm6OIc3XE12cBo6bVUB09+LvZt7lNXGCOqOd6J5OaU3Url+BNVPrZG/NqWLTTz8b687Bo21+mJa1DLZxICb2MTuGkokXG4H7Zr8HyM6W1LykfWShQf/o2PdoT54la/XBuPzz1XIppgGX+fKGkrxFBzw/Dbb8FfyrzMCvXo7xfgyZzpRbloHqlaE+rUrfvrnbNeqNDxx8sxUSFFJY+m3pP8w==; bm_sv=5247D5361ED7F464760DB27575E3D3D1~NAeI1AwynosACdbeChQSiDVKYcTx04aeu/i4losTc650rcNPTUdrhdWTZbfQ99q3/O2jLVTCfsHSG72jPmfz2AH54gHwey+wFRzwrBKDiGoF5gyFMEByVMFyuozv4JFDoh18f6sTIQbHaH4V5GXuiGDXf7LaLdTtQfOFHNmLP2w=; _abck=04D2CF35FC8971153BBDA9F8BF46734C~-1~YAAQJiWKUkOGI3J4AQAAeRgHqAWbg91Yg+GBB0DWS/TizQqM7n8kSmj3tKPqpQjNChgqtgEfKZn/4BAJaYhPy8mL5U3Gp94GKR+pJvSL58BYf1/rVM76cok6G9DPx7E2z8ZtsJCtv+Hop/muDHF11DuPqtIxWeuPJ/vMoAscVunPUgVP/hI4uzSJiOCLzr7hVa1Vxvffu3I7gZuxz04BDVZ/dGLPuNPrh2yiVwFpNArZfHV+Ei+UyD5vznismHZySoEhJmDjdJFMpZaq99lP9U6N3AV05Bd4y50E3XMy913MzxYugWV8DiyWVD8ikZ9fJMqtILCQ81u7AuY8EnNfR3CDJFMM2clAcQKkoz3QMoSoiAa3b1UB/97Z4y8YPuI6bBk8nLU=~0~-1~-1',
      'Range': 'bytes=0-'
    }

    response = requests.get (url, headers=headers).text
    
    soup = BeautifulSoup (response, 'lxml')

    video_data = json.loads (soup.find_all('script', {'type':'application/json'})[0].text)

    url_for_downloading = video_data ['props']['pageProps']['itemInfo']['itemStruct']['video']['downloadAddr']

    response = requests.get (url_for_downloading, headers=headers)

    if response.status_code == 206:
        print ('Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку')
    
    return response.content



def save_videos_audios_to_folder (nickname_stitch_needed):
    
    ''' Функция принимает на вход никнейм автора оригинального видео, 
    создаёт в папке проекта подпапку audio_video,
    сохраняет туда все видео, на которые вели собранные в Google-поиске ссылки,
    затем туда же сохраняет моно-wav-аудиофайлы этих видео. 
    
    Функция ничего не возвращает. Когда код закончит работу, раздаётся звук.'''
    
    path = os.path.join (os.getcwd(), nickname_stitch_needed)
    doc_path = os.path.join (path, f'{nickname_stitch_needed}_links.csv')

    all_links = []
    with open (doc_path, newline = '') as links:
        reader = csv.reader(links)
        all_links_as_lists = list (reader) # Здесь каждая строка как список из одной ссылки (одного элемента)
        for i in all_links_as_lists:
            all_links.append (i[0])
            
    path_upd = os.path.join (path, 'audio_video')

    try:
        os.mkdir (path_upd)
    except OSError:
        print ('Creation of the directory %s failed' % path)
    else:
        print ('Successfully created the directory %s' % path)

    print (f'Папка "audio_video" успешно создана в папке {nickname_stitch_needed} текущей директории')

    for i in tqdm (all_links, position=0, leave=True):
        
        try:

            aweme_id = get_aweme_id_final (i)

            content = video_response_for_downloading (i)
            open (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.mp4'), 'wb').write (content)

            my_clip = mp.VideoFileClip (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.mp4'))
            my_clip.audio.write_audiofile (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'))

            my_clip.close ()

            print (f'Всё прошло успешно! Проверьте аудио и видео №{all_links.index (i)+1}')

            sound = AudioSegment.from_wav (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'))
            sound = sound.set_channels (1)
            sound.export(os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'), format = 'wav')

            time.sleep (random.uniform (2, 6))
            
        except:
            print (f'ПРОБЛЕМА С ВИДЕО И АУДИО №{all_links.index (i)+1}. Пропускаем, вернитесь позже')
            pass

    winsound.Beep (440, 1000)
    print ('Поздравляем! Код завершил свою работу')
    
    
def save_videos_audios_to_folder_specified (nickname_stitch_needed, all_links):
    
    ''' Функция принимает на вход название нужной папки (nickname_stitch_needed) 
    и список ссылок на видео, для которых нужно собрать видео- и аудиофайлы, 
    создаёт в папке проекта подпапку audio_video, сохраняет туда все видео, 
    затем туда же сохраняет моно-wav-аудиофайлы этих видео. 
    
    Функция ничего не возвращает. Когда код закончит работу, раздаётся звук.'''
    
    path = os.path.join (os.getcwd(), nickname_stitch_needed)
    doc_path = os.path.join (path, f'{nickname_stitch_needed}_links.csv')
            
    path_upd = os.path.join (path, 'audio_video')

    try:
        os.mkdir (path_upd)
    except OSError:
        print ('Creation of the directory %s failed' % path)
    else:
        print ('Successfully created the directory %s' % path)

    print (f'Папка "audio_video" успешно создана в папке {nickname_stitch_needed} текущей директории')

    for i in tqdm (all_links, position=0, leave=True):
        
        try:

            aweme_id = get_aweme_id_final (i)

            content = video_response_for_downloading (i)
            open (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.mp4'), 'wb').write (content)

            my_clip = mp.VideoFileClip (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.mp4'))
            my_clip.audio.write_audiofile (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'))

            my_clip.close ()

            print (f'Всё прошло успешно! Проверьте аудио и видео №{all_links.index (i)+1}')

            sound = AudioSegment.from_wav (os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'))
            sound = sound.set_channels (1)
            sound.export(os.path.join (path_upd, f'{nickname_stitch_needed}_{aweme_id}.wav'), format = 'wav')

            time.sleep (random.uniform (2, 6))
            
        except:
            print (f'ПРОБЛЕМА С ВИДЕО И АУДИО №{all_links.index (i)+1}. Пропускаем, вернитесь позже')
            pass

    winsound.Beep (440, 1000)
    print ('Поздравляем! Код завершил свою работу')
    
    
def get_aweme_id_from_audio (filename):
    
    ''' Функция принимает на вход название аудиофайла из спарсенных видео
    и возвращает уникальный идентификатор видео (aweme_id).'''

    aweme_id = int (re.split ('_|.wav', filename)[1])
    return aweme_id



def stt_function (audio_file):

    ''' Для Python 3.9 и импортированной модели vosk!!!

    Функция принимает на вход название моно-wav-аудиофайла,
    расшифровывает речь на американском английском и
    записывает результат распознавания в txt-файл, название которого состоит
    из названия аудио файла + _stt.txt.
    
    Функция возвращает распознанный текст единой строкой. Код издаёт звук, когда завершает работу.'''

    # ЗАПУСК МОДЕЛИ РАСПОЗНАВАНИЯ РЕЧИ

    SetLogLevel(0)

    if not os.path.exists("model"):
        print(
            "Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
        exit(1)

    wf = wave.open (audio_file, "rb") # ЗДЕСЬ менять название аудио-файла
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        print ("Audio file must be WAV format mono PCM.")
        exit (1)

    model = Model("model")
    rec = KaldiRecognizer(model, wf.getframerate())

    final_json = {}
    i = 0

    with open(f'{audio_file[:-4]}_stt.txt', 'a') as output:  # ЗДЕСЬ заменять название файла для записи результата
        print('[', file=output)

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            with open(f'{audio_file[:-4]}_stt.txt', 'a') as output: # И ЗДЕСЬ
                print (rec.Result(), file = output)
                print(',', file=output)
        else:
            print(rec.PartialResult())

    with open(f'{audio_file[:-4]}_stt.txt', 'a') as output:  # И ЗДЕСЬ
        print (rec.FinalResult(), file = output)
        print (']', file = output)

    # СОХРАНЕНИЕ РЕЗУЛЬТАТА В ТЕКСТОВЫЙ ФАЙЛ
    # Мы перезаписываем тот же файл, в котором до этого делали предзапись результатов

    my_file = open (f'{audio_file[:-4]}_stt.txt', 'r') # И ЗДЕСЬ
    content = my_file.read()
    print (type (content))
    res = json.loads (content)
    final_string = ''
    for i in res:
        final_string = ' '.join ((final_string, i['text']))
    print (final_string)

    with open(f'{audio_file[:-4]}_stt.txt', 'w') as output:  # И ЗДЕСЬ
        print (final_string, file = output)
    
    winsound.Beep (440, 1000)

    return final_string



def unique_characteristics_for_common_videos (referer):
    
    ''' Функция позволяет собирать уникальную информацию о видео, как то:
    1. уникальный идентификатор видео (aweme_id)
    2. подпись к видео (description)
    3. время публикации в формате timestamp (timestamp_create_time)
    4. время публикации в человекочитаемом формате (type = string) (date)
    5. id автора видео (author_id)
    6. имя автора видео (author_name)
    7. никнейм автора (author_nickname)
    8. музыку в видео (music)
    9. лайки (likes)
    10. шеры (shares)
    11. комментарии (comments)
    12. просмотры (views)
    13. надписи на стикерах в видео (если есть) (sticker_text)
    
    Функция возвращает список этих элементов. 
    В списке на втором месте, после aweme_id, ссылка на это видео. '''
    
    aweme_id = get_aweme_id_final (referer) # id видео: взять из URL после video/, но до ?

    payload={}
    headers = {
      'authority': 'www.tiktok.com',
      'x-kl-ajax-request': 'Ajax_Request',
      'accept': 'application/json, text/plain, */*',
      'sec-ch-ua-mobile': '?0',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
      'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
      'sec-fetch-site': 'same-origin',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': referer,
      'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,el;q=0.6,de;q=0.5',
      'cookie': 'tt_webid_v2=6916212750071776773; tt_webid=6916212750071776773; ttwid=1%7CcdWyZ48EUgYhNscoWjjs4-x_r5maVNmW9iawlop2cNU%7C1610798758%7C6f5d0da3f04946eeaf6ced45118b69fd8ae2fea758073e919b5f6a01e8285087; passport_csrf_token=e5df83bd420baf05772ded41579ebfad; passport_csrf_token_default=e5df83bd420baf05772ded41579ebfad; store-idc=alisg; store-country-code=ru; cmpl_token=AgQQAPP2F-RO0o-gSj0Gth08xZrL6ZCRf4d9YPo--g; sid_guard=e6c07d4a41d789c3a181163cba66673e%7C1610804042%7C5184000%7CWed%2C+17-Mar-2021+13%3A34%3A02+GMT; uid_tt=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; uid_tt_ss=a4d5703ef83f07ca07cd0a278b14c4dbf85e372027e33d5605b4cb6018b791b9; sid_tt=e6c07d4a41d789c3a181163cba66673e; sessionid=e6c07d4a41d789c3a181163cba66673e; sessionid_ss=e6c07d4a41d789c3a181163cba66673e; MONITOR_WEB_ID=6916212750071776773; tt_csrf_token=GPy9U4CN50q1WNOtsFtAXWu-; s_v_web_id=verify_kk14bgxh_Ey7Ty0Nw_0w04_4iQX_8eBP_rzF6PEPNP7NM; csrf_session_id=a0f71447990040f7a19b35e99c980613; odin_tt=ba60c4b6907072ee26ad5b2462547eea1e16da5e658698dae644227ead9b99850d405faf8246f2d1e9087a40287e9af988a24c4ce98107ebed7edbbf8a3909636884ee832bd5e3252181f720a8b0512b'
    }

    response = requests.request("GET", referer, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'lxml') # загружаем DOM ссылки
    txt = soup.find_all('script', {'type':'application/json'})[0].text

    for_info = json.loads(txt)['props']['pageProps']['itemInfo']['itemStruct']
    aweme_id = for_info ['id']
    description = for_info ['desc']
    timestamp_create_time = for_info ['createTime']
    ts = time.gmtime(timestamp_create_time)
    date = time.strftime("%d.%m.%Y", ts)
    author_id = for_info ['author']['id']
    author_name = for_info ['author']['nickname']
    author_nickname = for_info ['author']['uniqueId']
    music = for_info ['music']['title']
    likes = for_info ['stats']['diggCount']
    shares = for_info ['stats']['shareCount']
    comments = for_info ['stats']['commentCount']
    views = for_info ['stats']['playCount']
    try:
        raw_sticker_text = for_info ['stickersOnItem'][0]['stickerText']
        sticker_text = ''
        for i in raw_sticker_text:
             sticker_text = ' '.join ((sticker_text, i))
    except IndexError:
        sticker_text = ''   
    row = [aweme_id, referer, description, timestamp_create_time, date, author_id, author_name, author_nickname, music,
          likes, shares, comments, views, sticker_text]
    return row

def csv_links_parse (all_links_to_parse, filename):
    
    ''' Функция принимает на вход список ссылок на не-ститч тикток-видео в формате csv и желаемое имя файлов.
    Возвращает:
    1. Файл с уникальными характеристиками видео
    2. Файл с комментариями к этим видео
    3. Файл с тэгами
    4. Граф тэгов
    
    Код издаёт звук, когда завершает работу.'''
    
    print ('Приступаем к парсингу уникальных характеристик видео')
    print ('Не переживайте, процесс может занять некоторое время')
    row_titles = ['aweme_id', 'link', 'description', 'timestamp_create_time', 'date', 'author_id', 'author_name', 
                  'author_nickname', 'music', 'likes', 'shares', 'comments', 'views', 'sticker_text']
    unique_char_table = pd.DataFrame (columns = row_titles)
    for i in tqdm (all_links_to_parse, position=0, leave=True):
        try:
            un_char = unique_characteristics_for_common_videos (i)
            unique_char_table.loc [len (unique_char_table)] = un_char # Добавляем ряд к датафрейму
            time.sleep (random.uniform(2, 6))
        except:
            print (f'Проблема с видео {i}, вернись к нему позже')
            pass
    unique_char_table ['hashtags'] = unique_char_table ['description'].apply (hashtags)
    unique_char_table.to_csv (f'{filename}_unique_characteristics.csv')
    print ('Записали уникальные характеристики видео в файл')
    
    print ('Приступаем к парсингу комментариев для видео')
    print ('Не переживайте, процесс может занять некоторое время')
    print ('Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,') 
    print ('сколько примерно комментариев под видео, которое парсится на данный момент.')
    print ('То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев')
    
#     comments_table = pd.DataFrame ()
#     for i in tqdm (all_links_to_parse, position=0, leave=True):
#         try:
#             video_comments = comments_parse_upd (i)
#             comments_table = comments_table.append (video_comments[1], ignore_index = True)
#         except:
#             print (f'Проблема с видео {i}, вернись к нему позже')
#             pass            
#     comments_table_to_save = clear_comments_table_user_dict_without_saving (comments_table)
#     comments_table_to_save.to_csv (f'{filename}_comments_table.csv', sep = ';')
#     print ('Почистили таблицу комментариев и записали в файл')
    
    print ('Приступаем к сбору тэгов из подписей к видео')
    with open (f'{filename}_tags.csv', 'w+', newline = '', encoding = 'utf-8') as tags:
        writer = csv.writer (tags, delimiter = ",")
        writer.writerow (['original_hashtag', 'relative_hashtag', 'source'])
        for i in unique_char_table ['hashtags']:
            for j in i:
                print (j)
                writer.writerow ([filename, drop_hashtag (j), 'caption'])
            for a in range (len (i)):
                for b in range (a + 1, len (i)):
                    writer.writerow ([drop_hashtag (i[a]), drop_hashtag (i[b]), 'caption_internal_links'])
    print ('Записали вторую часть тэгов в файл')
    
    print ('Превращаем csv-файл тэгов в gexf-файл для создания графа')
    tags_to_gexf = pd.read_csv (f'{filename}_tags.csv', encoding = 'utf-8')
    tags_to_gexf = tags_to_gexf.drop ('source', axis = 1)
    tags_to_gexf.columns = ['source', 'target']
    gexf_graph = nx.from_pandas_edgelist (tags_to_gexf, create_using = nx.MultiGraph ())
    nx.write_gexf (gexf_graph, f'{filename}_graph.gexf')
    print ('Gexf-файл успешно сохранён')
    
    winsound.Beep (440, 1000)

    print ('Ура! Код завершил работу. Проверьте Ваши файлы')

def csv_links_parse_comments_only (all_links_to_parse, filename):
    
    ''' Функция принимает на вход список ссылок на не-ститч тикток-видео в формате csv и желаемое имя файлов.
    Возвращает файл с комментариями к этим видео. Код издаёт звук, когда завершает работу. '''
    
    print ('Приступаем к парсингу комментариев для видео')
    print ('Не переживайте, процесс может занять некоторое время')
    print ('Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,') 
    print ('сколько примерно комментариев под видео, которое парсится на данный момент.')
    print ('То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев')
    
    comments_table = pd.DataFrame ()
    for i in tqdm (all_links_to_parse, position=0, leave=True):
        try:
            video_comments = comments_parse_upd (i)
            comments_table = comments_table.append (video_comments[1], ignore_index = True)
        except TypeError:
            print ('Либо у этого видео нет комментариев, либо произошла какая-то ошибка')
            pass
    comments_table_to_save = clear_comments_table_user_dict_without_saving (comments_table)
    comments_table_to_save.to_csv (f'{filename}_comments_table.csv', sep = ';')
    print ('Почистили таблицу комментариев и записали в файл')
    
    winsound.Beep (440, 1000)

    print ('Ура! Код завершил работу. Проверьте файл')

def tags_from_unchar (filename):
    
    ''' Функция принимает на вход csv-файл с уникальными характеристиками видео,
    вычленяет тэги из подписей к видео, группирует их попарно внутри одной подписи, 
    тем самым выстраивая связь между тэгами, перезаписывает файл с тэгами, добавляя новые ряды,
    и сохраняет результат в виде графа тэгов как gexf-файл.'''
    
    print ('Приступаем к сбору тэгов из подписей к видео')
    unique_char_table = pd.read_csv (f'{filename}_unique_characteristics.csv').drop ('Unnamed: 0', axis = 1)
    with open (f'{filename}_tags.csv', 'w+', newline = '', encoding = 'utf-8') as tags:
        writer = csv.writer (tags, delimiter = ",")
        writer.writerow (['original_hashtag', 'relative_hashtag', 'source'])
        for i in unique_char_table ['hashtags']:
            reading_as_list = ast.literal_eval (i)
            for j in reading_as_list:
                print (j)
                writer.writerow ([filename, drop_hashtag (j), 'caption'])
            for a in range (len (reading_as_list)):
                for b in range (a + 1, len (reading_as_list)):
                    writer.writerow ([drop_hashtag (reading_as_list[a]), drop_hashtag (reading_as_list[b]), 
                                      'caption_internal_links'])
    print ('Записали вторую часть тэгов в файл')
    
    print ('Превращаем csv-файл тэгов в gexf-файл для создания графа')
    tags_to_gexf = pd.read_csv (f'{filename}_tags.csv', encoding = 'utf-8')
    tags_to_gexf = tags_to_gexf.drop ('source', axis = 1)
    tags_to_gexf.columns = ['source', 'target']
    gexf_graph = nx.from_pandas_edgelist (tags_to_gexf, create_using = nx.MultiGraph ())
    nx.write_gexf (gexf_graph, f'{filename}_graph.gexf')
    print ('Gexf-файл успешно сохранён')
    
    winsound.Beep (440, 1000)

    print ('Ура! Код завершил работу. Проверьте Ваши файлы')
    
def scroll (driver, timeout):
    
    ''' Функция скроллит страницу до конца '''
    
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

def all_links (url):
    
    ''' Функция собирает все ссылки на странице '''
    
    # Setup the driver. This one uses firefox with some options and a path to the geckodriver
    driver = wb.Chrome('C:/Users/User/Documents/Практика/chromedriver_win32/chromedriver.exe')
    # implicitly_wait tells the driver to wait before throwing an exception
    driver.implicitly_wait(30)
    # driver.get(url) opens the page
    driver.get(url)
    # This starts the scrolling by passing the driver and a timeout
    scroll(driver, 5)
    # Once scroll returns bs4 parsers the page_source
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    # Them we close the driver as soup_a is storing the page source
    driver.close()

    # Empty array to store the links
    links = []

    # Looping through all the a elements in the page source
    for link in soup_a.find_all('a'):
        # link.get('href') gets the href/url out of the a element
        links.append(link.get('href'))

    return links

def creator_links (nickname_creator_needed):
    
    ''' Функция принимает на вход никнейм пользователя, ссылки из аккаунта которого нужно собрать, 
    и возвращает список ссылок на все видео в аккаунте. Также сохраняет ссылки в csv-файл.'''
    
    print ('Вручную реши капчу!')
    nickname_creator_main_link = 'https://www.tiktok.com/@' + f'{nickname_creator_needed}'
    nickname_creator_all_links = all_links (nickname_creator_main_link)
    all_clear_links = []
    for i in nickname_creator_all_links:
        if len (re.findall ('https://www.tiktok.com/@' + nickname_creator_needed + '/video/.*\d', nickname_creator_all_links [nickname_creator_all_links.index(i)]))>0:
            all_clear_links.append (i)
    with open (f'{nickname_creator_needed}_links.csv', 'w', newline = '') as links:
        writer = csv.writer (links, delimiter = ",")
        for i in all_clear_links:
            writer.writerow (i.split())
    print ('Сохранили все ссылки в файл')
    
    return all_clear_links

def mobile_links_to_normal (filename):
    
    ''' Функция принимает на вход список ссылок на видео, скопированных из мобильного приложения, 
    в формате "https://vm.tiktok.com/..." и возвращает список ссылок ПК-типа "www.tiktok.com/@никнейм/video/..."
    Если функция не работает, возможно, условия доступа к серверной части TikTok вновь изменились, 
    и следует переписать функцию, потому что ответ переводит на капчу.'''
    
    all_links = []
    with open (filename, newline = '', encoding = 'utf-8-sig') as links: 
        # utf-8-sig вместо utf-8 помогает избежать символов начала документа в первой строке
        reader = csv.reader(links)
        all_links_as_lists = list (reader) # Здесь каждая строка как список из одной ссылки (одного элемента)
        for i in all_links_as_lists:
            all_links.append (i[0])
    all_normal_links = []
    for every_url in tqdm (all_links):
        response = requests.get (every_url)
        soup = BeautifulSoup (response.text)
        normal_url = soup.find_all ('link')[0]['href']
        all_normal_links.append (normal_url)
    return all_normal_links

def tags_parsing (tag_needed):
    
    ''' Для работы функции нужно заранее закачать в папку проекта html-копию нужной страницы тэга в формате "тэг Hashtag.html",
    то есть нужно предварительно в браузере вручную проскроллить нужную страницу вниз, сохранить её, 
    убрать лишние символы из названия (в том числе первый знак #), а затем загрузить в Jupyter Notebook.
    Функция принимает на вход название тэга (например, "easterneuropean") как строку, 
    создаёт папку (название папки совпадает с названием тэга) и возвращает список всех найденных в html-файле ссылок на видео,
    а также сохраняет эти ссылки в csv-файл.'''
    
    with open (f'{tag_needed} Hashtag.html', 'r', encoding = 'utf-8') as f:
        contents = f.read()

        soup = BeautifulSoup(contents, 'lxml')
    
    all_links = []
    for i in soup.find_all ('a', href=True):
        if 'video' in i['href']:
            print (i ['href'])
            all_links.append (i ['href'])
    try:
        path = os.path.join (os.getcwd(), tag_needed)
        os.mkdir (path)
    except OSError:
        print ('Creation of the directory %s failed' % path)
    else:
        print ('Successfully created the directory %s' % path)
    print (f'Папка {tag_needed} успешно создана в текущей директории')

    with open (os.path.join (path, f'{tag_needed}_links.csv'), 'w', newline = '') as links:
        writer = csv.writer (links, delimiter = ",")
        for i in all_links:
            writer.writerow (i.split())
    return all_links

def get_folders_name (filename):
    
    ''' Функция принимает на вход название файла и вычленяет оттуда название соответствующей папки.'''
    
    x = filename.split ('_')
    return x[0]

# Сбор #Stitch-видео

Пример сбора видео-ответов на видео аккаунта "kallmekris".

In [2]:
stitches_parsing_workflow ()

Введите ник пользователя, для видео которого нужны ститчи: kallmekris
Successfully created the directory C:\Users\User\TikTok\kallmekris
Папка kallmekris успешно создана в текущей директории
Начинаем сбор ссылок на видео и тэги из Google-поиска
Стартовая позиция:  0
Загрузили
#stitch с @kallmekris у вас тоже так было?       - TikTok
https://www.tiktok.com/@tilitili111/video/6894974043733101826
#stitch with @tabithalipkin Burn it all down     - TikTok
https://www.tiktok.com/@kallmekris/video/6926224537558207750
#stitch with @morymcfly3 Was genuinely not expecting that - TikTok
https://www.tiktok.com/@kallmekris/video/6902077430257798402
#stitch with @kallmekris @nikkietutorials - TikTok
https://www.tiktok.com/@jeenie.weenie/video/6890298467705457922
#stitch with @kallmekris please don't be tagging Kris in this, thank you!
https://www.tiktok.com/@dr_inna/video/6906596814707756294
#stitch with @kallmekris this is in no way a criticism of Kris (or ...
https://www.tiktok.com/@dr_inna/video/

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

Загрузили
Код завершил свою работу :)
Собрали ссылки на видео и тэги из Google-поиска
Выделяем из результатов тэги
Записали первую часть тэгов в файл
Выделяем все ссылки на видео для парсинга


  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

Сохранили все ссылки в файл
Приступаем к парсингу уникальных характеристик видео
Не переживайте, процесс может занять некоторое время


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:38<00:00,  4.29s/it]


Записали уникальные характеристики видео в файл
Приступаем к сбору тэгов из подписей к видео
#YouGotThis
#exprESSIEyourself
Записали вторую часть тэгов в файл
Превращаем csv-файл тэгов в gexf-файл для создания графа
Gexf-файл успешно сохранён
Ура! Код завершил работу. Проверьте Ваши файлы


In [3]:
save_videos_audios_to_folder ('kallmekris')

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

Successfully created the directory C:\Users\User\TikTok\kallmekris
Папка "audio_video" успешно создана в папке kallmekris текущей директории
Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку


chunk:  39%|███████████████████████▌                                     | 141/365 [00:00<00:00, 1396.06it/s, now=None]

MoviePy - Writing audio in C:\Users\User\TikTok\kallmekris\audio_video\kallmekris_6946196243940969730.wav



  0%|                                                                                            | 0/9 [00:01<?, ?it/s]

...

MoviePy - Writing audio in C:\Users\User\TikTok\kallmekris\audio_video\kallmekris_6890298467705457922.wav



 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [00:48<00:05,  5.42s/it]

MoviePy - Done.
Всё прошло успешно! Проверьте аудио и видео №9


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:52<00:00,  5.88s/it]


Поздравляем! Код завершил свою работу


# Сбор видео из аккаунта

На примере аккаунта simplebiologist.

In [5]:
collected_links = creator_links ('simplebiologist')

Вручную реши капчу!
Сохранили все ссылки в файл


In [6]:
csv_links_parse (collected_links, 'simplebiologist')

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Приступаем к парсингу уникальных характеристик видео
Не переживайте, процесс может занять некоторое время


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:49<00:00,  4.38s/it]


Записали уникальные характеристики видео в файл
Приступаем к парсингу комментариев для видео
Не переживайте, процесс может занять некоторое время
Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,
сколько примерно комментариев под видео, которое парсится на данный момент.
То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев
Приступаем к сбору тэгов из подписей к видео
#friendsinhighplaces
#spidersarefriends
#fyp
#science
#todayilearned
#bio
#funfact
#science
#storytime
#bedbugs
#fyp
#nothanks
#science
#ants
#fyp
#toomany
#funfact
#science
#storytime
#fyp
#rainbowconnection
#science
#covid19
#answeringquestions
#fyp
#covid19
#vaccine
#fyp
#science
#moon
#fyp
#todayilearned
#science
#fyp
#storytime
#stamps
#physics
#fyp
#science
#storytime
#worry
#vaccine
#science
#dramatic
#parody
#fyp
#science
#storytime
#fyp
#birds
#birdsarentreal
#honeymoon
#romantic
#covid
#covid19
#nosmell
#science
#scienceismagic
#brainpowe

Создайте вручную папку по названию аккаунта и перенесите туда созданные папки.

In [8]:
save_videos_audios_to_folder ('simplebiologist')

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Successfully created the directory C:\Users\User\TikTok\simplebiologist
Папка "audio_video" успешно создана в папке simplebiologist текущей директории
Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку


chunk:  22%|█████████████▎                                               | 153/704 [00:00<00:00, 1485.45it/s, now=None]

MoviePy - Writing audio in C:\Users\User\TikTok\simplebiologist\audio_video\simplebiologist_6956704071315229957.wav



  0%|                                                                                           | 0/25 [00:01<?, ?it/s]

...

MoviePy - Writing audio in C:\Users\User\TikTok\simplebiologist\audio_video\simplebiologist_6889097710201212165.wav



 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [02:52<00:07,  7.12s/it]

MoviePy - Done.
Всё прошло успешно! Проверьте аудио и видео №25


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [02:57<00:00,  7.10s/it]


Поздравляем! Код завершил свою работу


# Сбор по тэгу

Пример по тэгу #whatdotheylearn.

Предварительно откройте страницу с тэгом (https://www.tiktok.com/tag/whatdotheylearn), проскролльте её до конца/ до нужного места, кликните правой кнопкой мыши, "Сохранить как", сохраните на компьютер, отформатировав название файла от исходного "#whatdotheylearn Hashtag Videos on TikTok.html" до "whatdotheylearn Hashtag.html". Загрузите в Jupyter Notebook.

In [9]:
collected_links = tags_parsing ('whatdotheylearn')

https://www.tiktok.com/@robin_d_show/video/6919958888049609986
https://www.tiktok.com/@mrsewald/video/6947857113985076485
Successfully created the directory C:\Users\User\TikTok\whatdotheylearn
Папка whatdotheylearn успешно создана в текущей директории


In [10]:
csv_links_parse (collected_links, 'whatdotheylearn')

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Приступаем к парсингу уникальных характеристик видео
Не переживайте, процесс может занять некоторое время


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.87s/it]


Записали уникальные характеристики видео в файл
Приступаем к парсингу комментариев для видео
Не переживайте, процесс может занять некоторое время
Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,
сколько примерно комментариев под видео, которое парсится на данный момент.
То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев
Приступаем к сбору тэгов из подписей к видео
#usa
#americans
#germany
#helpme
#whatdotheylearn
#forreal
#fyp
#whatdotheylearn
#trendi
Записали вторую часть тэгов в файл
Превращаем csv-файл тэгов в gexf-файл для создания графа
Gexf-файл успешно сохранён
Ура! Код завершил работу. Проверьте Ваши файлы


In [11]:
save_videos_audios_to_folder ('whatdotheylearn')

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Successfully created the directory C:\Users\User\TikTok\whatdotheylearn
Папка "audio_video" успешно создана в папке whatdotheylearn текущей директории
Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку


chunk:  13%|███████▋                                                    | 153/1190 [00:00<00:00, 1466.21it/s, now=None]

MoviePy - Writing audio in C:\Users\User\TikTok\whatdotheylearn\audio_video\whatdotheylearn_6919958888049609986.wav



  0%|                                                                                            | 0/2 [00:02<?, ?it/s]

...

Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку


 50%|██████████████████████████████████████████                                          | 1/2 [00:07<00:05,  5.78s/it]

MoviePy - Writing audio in C:\Users\User\TikTok\whatdotheylearn\audio_video\whatdotheylearn_6947857113985076485.wav
MoviePy - Done.
Всё прошло успешно! Проверьте аудио и видео №2


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.55s/it]


Поздравляем! Код завершил свою работу


# Сбор по вручную собранным ссылкам

Для начала подготовьте csv-файл со списком ссылок, где каждая ссылка будет на отдельной строчке. Cписок будет выглядеть примерно так:
* https://www.tiktok.com/@tuckerbudzyn/video/6957318988678335749
* https://www.tiktok.com/@lactose_intolerable/video/6956505664675958017
* https://www.tiktok.com/@mad__magic/video/6924405715364793606

Для примера мы назвали файл 'Random_links.csv'.

In [14]:
collected_links = []
with open ('Random_links.csv', newline = '', encoding = 'utf-8-sig') as links: 
    # utf-8-sig вместо utf-8 помогает избежать символов начала документа в первой строке
    reader = csv.reader(links)
    all_links_as_lists = list (reader) # Здесь каждая строка как список из одной ссылки (одного элемента)
    for i in all_links_as_lists:
        collected_links.append (i[0])

In [15]:
csv_links_parse (collected_links, 'Random')

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Приступаем к парсингу уникальных характеристик видео
Не переживайте, процесс может занять некоторое время


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.56s/it]


Записали уникальные характеристики видео в файл
Приступаем к парсингу комментариев для видео
Не переживайте, процесс может занять некоторое время
Сложив числа между двумя плашками прогресса, вы можете навскидку сказать,
сколько примерно комментариев под видео, которое парсится на данный момент.
То есть если вы видите: "50 Это не конец 49 Это конец", то у видео больше 50, но меньше 100 комментариев
Приступаем к сбору тэгов из подписей к видео
#runawaychallenge
#dogsoftiktok
#fyp
#australia
#normativemalealexithymia
#toxicmasculinity
#wintermagic
#crows
#witchfamiliar
#familiarspirit
#babywitchtok
#cottagecore
#zodiacsign
#fyp
#HowTo
#animalfriends
#mirroring
Записали вторую часть тэгов в файл
Превращаем csv-файл тэгов в gexf-файл для создания графа
Gexf-файл успешно сохранён
Ура! Код завершил работу. Проверьте Ваши файлы


Создайте вручную папку по названию аккаунта и перенесите туда созданные папки.

In [18]:
save_videos_audios_to_folder ('Random')

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Creation of the directory C:\Users\User\TikTok\Random failed
Папка "audio_video" успешно создана в папке Random текущей директории
Видео найдено, всё хорошо! Начинаем загрузку видео и аудио в папку


chunk:  23%|█████████████▉                                               | 153/669 [00:00<00:00, 1485.45it/s, now=None]

MoviePy - Writing audio in C:\Users\User\TikTok\Random\audio_video\Random_6957318988678335749.wav



  0%|                                                                                            | 0/3 [00:02<?, ?it/s]

...

MoviePy - Writing audio in C:\Users\User\TikTok\Random\audio_video\Random_6924405715364793606.wav



 67%|████████████████████████████████████████████████████████                            | 2/3 [00:16<00:07,  7.48s/it]

MoviePy - Done.
Всё прошло успешно! Проверьте аудио и видео №3


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:20<00:00,  6.88s/it]


Поздравляем! Код завершил свою работу


Автор кода: Мария Казакова (@undine_su_menulio), Москва, 2021.

Контакты: marikasakowa@gmail.com